In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
w2v=KeyedVectors.load("/content/drive/MyDrive/FYP/IMDb/FastText/ft")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [ ]:
import tensorflow as tf
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   19,  7608,  7149, ...,   123,  4013,   486],
       [    4,   203,     2, ...,  1900,    75,   216],
       [   10,   132,     3, ...,    67,    16,   333],
       ...,
       [ 3880,    70,   148, ..., 22558,     2,  5887],
       [   15,     1,   341, ...,    49,   695,    44],
       [    9,   154,    18, ...,   759,    11,    18]], dtype=int32)

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
d={}
for word,i in tokenizer.word_index.items():
  d[i]=word

In [ ]:
for i in X:
  for j in range(len(i)):
    if i[j]==0:
      pass
    else:
      p=d[i[j]]
      if p in w2v:
        i[j]=sum(w2v[p])
      else:
        i[j]=0
  print(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Streaming output truncated to the last 5000 lines.
  -37   17  -21    8  104   -3   -1   65   16   -2   37   65  -35   40
   31   40   65   31    5    4    2   -1   39  -33  -17  -51   78  -35
  -19   -1   40  -24  -17   -8   25   14   31   10]
[  24   50   65   14  -11  -51  -42   62  -35    3  -18  -37   19   14
  -51   44   12  -18  -25   14  -51   -4   15  -42  138    3  -26   50
   65   -6    2   31    6  -51   16  138    7 -149    9  -26   50   31
    0   16   15   -9  138  -86   15   -6    0    2  -26   50  -76    7
  138   16   15   13  -23   -7  138   73   15   15]
[ -83   31  -11   65   19   18  -12  -32  138   18   30  -35   15   19
  -51  175    0   31  138   -2  -33   65  -33   20   14  -31   31   53
  -16  138  -37   19  -17  -51   78 -128   37  -11   15  -41  -62   19
   19  -26  -94  -34  -34  138  -32   19    5  -26  -62  -15   14  -35
   20    0    6  -36   -2  -51  -51   31    4   -8]
[ 138  -86   15  -51   81  -33    0   92   19    0   15  -26   14   65
 -128   14  

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [ ]:
display_metrics(y_test,y_pred)

0.55352
0.5534739070761423
0.553848608579975
0.5537937987216677


In [ ]:
from sklearn import svm 
clf = svm.SVC(kernel='linear',verbose=1,max_iter=10000)
clf.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=10000, verbose=1)

In [ ]:
y_pred=clf.predict(X_test)
display_metrics(y_test,y_pred)

0.50616
0.48839972355120054
0.5040117366732345
0.5034990403379169


In [ ]:
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred=DTclf.predict(X_test)
display_metrics(y_test,y_pred)

0.53384
0.5338119272731119
0.5338352522589194
0.5338427316681267
